
## **Intent Classification with Orq.ai**

This quick guide demonstrates how to build and evaluate an AI application for intent classification using the Orq platform.

Before you begin, make sure you have an Orq account. If you don't have one, sign up first. Additionally, we’ve prepared a Google Colab file that you can copy and run immediately, simplifying the setup process. Just replace the API key, and you’re ready to go! After completing this tutorial, visit the Orq documentation for more advanced topics

**Why Intent Classification is Useful**  
Intent classification is a powerful tool for understanding and categorizing user inputs, making it essential for building smarter, more responsive applications. Here are a few use cases:

- Customer Support: Automatically categorize support tickets or chatbot queries to route them to the right team.
- E-commerce Personalization: Understand user intents like "buy," "browse," or "compare" to tailor shopping experiences.
- Task Automation: Trigger specific workflows based on user commands, such as scheduling meetings or setting reminders.

**Step 1: Install Dependencies**  
Start by installing the required packages. These include the Orq SDK and additional libraries for handling datasets and managing environment variables.

In [ ]:
pip install orq-ai-sdk datasets huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for orq-ai-sdk: filename=orq_ai_sdk-2.13.4-py3-none-any.whl size=23578 sha256=7fc5cb73cc3ace91ace52e3ab9a882f276a700d3e4206c217ed83e3106ab6220
  Stored in directory: /root/.cache/pip/wheels/24/a7/7d/823d25efb69295bec7a2355ad01cea27f2c3fb4e3adc9b2d28
Successfully built orq-ai-sdk


**Step 2: Initialize the Orq Client**  

The Orq client allows you to communicate with the Orq platform. Set it up using your API key, which can be stored as an environment variable (ORQ_API_KEY) or passed directly.

In [ ]:
import os

from orq_ai_sdk import Orq

client = Orq(
  api_key=os.environ.get("ORQ_API_KEY", "insert_API_key_here"),
)

### Hugging Face

Before proceeding, sign up for a free Hugging Face account if you don’t already have one. You’ll need an API key to access their datasets library. Retrieve your API key here after signing up or logging in.

**Step 3: Load a Dataset**  
Use the Hugging Face datasets library to load the dataset for intent classification. Here, we use a public dataset that contains user queries labeled with intents.

In [ ]:
from huggingface_hub import login

# Use your Hugging Face API token
login(token="insert_HF_API_key_here")

In [ ]:
# Convert to a pandas DataFrame (selecting the "train" split as an example)
df = ds["train"].to_pandas()

# Select the top 300 rows
df = df.head(50)

# Display the DataFrame or save it
print(df)

                                                 text                intent
0    listen to westbam alumb allergic on google music             PlayMusic
1          add step to me to the 50 clásicos playlist         AddToPlaylist
2   i give this current textbook a rating value of...              RateBook
3                play the song little robin redbreast             PlayMusic
4   please add iris dement to my playlist this is ...         AddToPlaylist
5   add slimm cutta calhoun to my this is prince p...         AddToPlaylist
6                 i want to listen to seventies music             PlayMusic
7                                        Good Morning             Greetings
8               play a popular chant by brian epstein             PlayMusic
9                                     find fish story  SearchScreeningEvent
10                            book a spot for 3 in mt        BookRestaurant
11  i need a forecast for jetmore  massachusetts i...            GetWeather
12          

In [ ]:
# Extract unique labels from the "intent" column
unique_labels = df["intent"].unique()
unique_labels

array(['PlayMusic', 'AddToPlaylist', 'RateBook', 'Greetings',
       'SearchScreeningEvent', 'BookRestaurant', 'GetWeather',
       'Book Meeting', 'SearchCreativeWork'], dtype=object)

### **Adding Generated Responses to DataFrame**

This code adds an empty column to the DataFrame for storing generated responses, iterates through each row to generate a response based on the `text_input`, and then saves the updated DataFrame.

#### **Intent classification prompt**

This deployment is designed to classify user inputs into specific intents to understand the purpose behind their requests. The model will identify the most appropriate intent from a predefined set, enabling precise and context-aware responses.

Intent recognition is particularly useful when setting up chatbots

This is the prompt in Orq.ai:

```plaintext
You are tasked with identifying the intent behind user inputs based on the following intents:
'PlayMusic', 'AddToPlaylist', 'RateBook', 'Greetings', 'SearchScreeningEvent', 'BookRestaurant', 'GetWeather', 'Book Meeting', 'SearchCreativeWork', 'Cancellation', 'Affirmation', 'excitment'.

Here are some examples:

Input: "Hey there, how are you doing?"
Intent: Greetings

Input: "Play the album Abbey Road by The Beatles."
Intent: PlayMusic

Input: "Add this song to my workout playlist."
Intent: AddToPlaylist

Input: "This book deserves a solid five-star rating."
Intent: RateBook

ONLY OUTPUT THE LABEL WITHOUT ''

here is the input that needs intent classification: {{text}}



**Step 4: Invoke Orq Deployment**  

Integrate the Orq intent classification model by invoking a deployed model for predictions. Iterate through the dataset and store the results in a new column called "output".

In [ ]:
# Placeholder for the deployment key
deployment_key = "intent_classification"

# Create an empty list to store outputs
outputs = []

# Iterate through the rows of the DataFrame
for index, row in df.iterrows():
    text_input = row["text"]  # Extract the text column

    # Invoke the model for each text input
    generation = client.deployments.invoke(
        key=deployment_key,
        context={
            "environments": []
        },
        inputs={
            "text": text_input
        },
        metadata={
            "custom-field-name": "custom-metadata-value"
        }
    )

    # Append the output to the list
    outputs.append(generation.choices[0].message.content)


KeyboardInterrupt: 

In [ ]:
 # Add the output column to the DataFrame
df["output"] = outputs

**Step 5: Evaluate Model Performance**  

Use metrics such as accuracy, precision, recall, and F1-score to assess the quality of your model's predictions.

In [ ]:

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

true_labels = df["intent"]
predicted_labels = df["output"]

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Print the metrics
print("Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Performance Metrics:
Accuracy: 0.9400
Precision: 0.9500
Recall: 0.9491
F1 Score: 0.9484


**Next Steps**  

Congratulations! You've successfully built and evaluated an intent classification application using Orq. To further enhance your application:

- Explore other datasets and use cases.
- Integrate the model into a chatbot or voice assistant.
- Automate deployment and testing with Orq’s advanced features.

For more resources, visit the Orq documentation.